In [1]:
# Data handling
import pandas as pd

# Beta analysis
from skbio.diversity import beta_diversity
from skbio.stats.composition import clr
import numpy as np
from skbio.stats.distance import DistanceMatrix
from scipy.spatial.distance import pdist, squareform

# Statistical tesing
from skbio.stats.distance import permanova

In [2]:
# Importing data
import_str = "/ceph/projects/179_Oncdon/shawn.loo/workspace/"

V107_data = pd.read_csv(f"{import_str}V107_maindataset.csv")
E975_data = pd.read_csv(f"{import_str}E975_maindataset.csv")

In [3]:
V107_data[V107_data["abundance"] > 1]

,genus,phylum,sample_id,abundance,group,week,pt_identifier
8,Alistipes,Bacteroidota,179supB00007a,15.02526,R,week0,MAID-0728
12,Anaerobutyricum,Firmicutes,179supB00007a,1.96002,R,week0,MAID-0728
19,Anaerostipes,Firmicutes,179supB00007a,1.43319,R,week0,MAID-0728
29,Bacteroides,Bacteroidota,179supB00007a,2.68112,R,week0,MAID-0728
33,Bifidobacterium,Actinobacteria,179supB00007a,4.82403,R,week0,MAID-0728
...,...,...,...,...,...,...,...
105143,Lachnoclostridium,Firmicutes,179fece00063a,24.49536,NR,week6,MAID-1101.1
105148,Lacticaseibacillus,Firmicutes,179fece00063a,1.63017,NR,week6,MAID-1101.1
105213,Phocaeicola,Bacteroidota,179fece00063a,1.54947,NR,week6,MAID-1101.1
105222,Pseudoramibacter,Firmicutes,179fece00063a,2.16858,NR,week6,MAID-1101.1


In [4]:
# Prepare table into OTU format
def create_OTU(df):

    result_tbl = df.pivot_table(
        index = "sample_id",
        columns = "genus",
        values = "abundance",
        aggfunc = "sum",
        fill_value = 0
    )
    
    return result_tbl

# Creating OTU table
V107_otu = create_OTU(V107_data)
E975_otu = create_OTU(E975_data)

# Building metadata for plotting
def create_metadata(df):
    
    metadata = df.drop_duplicates(subset = ["sample_id"])
    metadata = metadata[["sample_id", "group", "week", "pt_identifier"]]
    metadata.set_index("sample_id", inplace = True)
    
    return metadata

# Create metadata
V107_metadata = create_metadata(V107_data)
E975_metadata = create_metadata(E975_data)

In [5]:
# Saving for R
V107_metadata.to_csv("V107_metadata.csv")
E975_metadata.to_csv("E975_metadata.csv")

### Bray-Curtis Calculation

In [6]:
V107_bray_curtis_df = beta_diversity("braycurtis", V107_otu.values, V107_otu.index)
E975_bray_curtis_df = beta_diversity("braycurtis", E975_otu.values, E975_otu.index)

### Jaccard Calculation

In [7]:
# Convert to boolean value
V107_bool_df = (V107_otu > 0).astype(int)
E975_bool_df = (E975_otu > 0).astype(int)

In [8]:
V107_jaccard = beta_diversity("jaccard", V107_bool_df.values, V107_otu.index)
E975_jaccard = beta_diversity("jaccard", E975_bool_df.values, E975_otu.index)

### Aitchison Calculation

In [9]:
def add_psuedocount_clr(df):

    dataset = df.copy()

    # Loop through each row
    for i in dataset.index:
        
        # Retrieve values
        row_val = dataset.loc[i].values
        
        # Get nonzero values
        nonzero_row_val = row_val[row_val > 0]
        
        # Retrieve pseudo value
        psuedo_val = 0.5 * nonzero_row_val.min()
        
        # Replace zero with psuedo value
        row_val[row_val == 0] = psuedo_val
        
        # normalize
        dataset.loc[i] = row_val / row_val.sum()
        
    # Perform CLR
    clr_dataset = pd.DataFrame(clr(dataset), index = dataset.index, columns = dataset.columns)
    
    return clr_dataset

V107_clr = add_psuedocount_clr(V107_otu)
E975_clr = add_psuedocount_clr(E975_otu)

In [10]:
# Perform Aitchison calculation (manually), skbio's version expects raw counts

def aitchison_distance_cal(df):
    
    # Compute pairwise distance
    distance = pdist(df.values, metric = "euclidean")
    
    # Convert to matrix form
    matrix = squareform(distance)
    
    # return skbio distance matrix object
    result_matrix = DistanceMatrix(matrix, ids = df.index)
    
    return result_matrix

V107_aitchison = aitchison_distance_cal(V107_clr)
E975_aitchison = aitchison_distance_cal(E975_clr)

### Statistical Test

Done in RStudio

In [11]:
# Convert to csv friendly format to export
def convert_to_df(mat):
    
    dataset = pd.DataFrame(mat.data, index = mat.ids, columns = mat.ids)
    
    return dataset

In [12]:
V107_bray_curtis_df = convert_to_df(V107_bray_curtis_df)
E975_bray_curtis_df = convert_to_df(E975_bray_curtis_df)

V107_jaccard = convert_to_df(V107_jaccard)
E975_jaccard = convert_to_df(E975_jaccard)

V107_aitchison = convert_to_df(V107_aitchison)
E975_aitchison = convert_to_df(E975_aitchison)

In [13]:
# Saving them for R
V107_bray_curtis_df.to_csv("V107_bray_curtis.csv")
E975_bray_curtis_df.to_csv("E975_bray_curtis.csv")

V107_jaccard.to_csv("V107_jaccard.csv")
E975_jaccard.to_csv("E975_jaccard.csv")

V107_aitchison.to_csv("V107_aitchison.csv")
E975_aitchison.to_csv("E975_aitchison.csv")